In [19]:
import cv2 as cv
import numpy as np

In [20]:
cap = cv.VideoCapture("pedestrian.mp4") # Videoyu aç, pedestrian.mp4 dosyasını oku

roi; bir bölgeye odaklanmak için kullanılır

In [21]:
fgbg = cv.createBackgroundSubtractorMOG2(history=500, 
                                         varThreshold=100) # Arka plan çıkarma için MOG2 algoritmasını oluştur, 
                                                           # geçmiş uzunluğu ve hassaslık ayarı

In [22]:
def process(image): # Görüntü işleme fonksiyonu
    mask = fgbg.apply(image) # Hareketli nesneleri tespit etmek için foreground maskesi uygula
    line = cv.getStructuringElement(cv.MORPH_RECT,
                                    (1, 5),
                                    (-1, -1)) # Dikdörtgen bir morfolojik element oluştur
    mask = cv.morphologyEx(mask,
                           cv.MORPH_OPEN,
                           line) # Gürültüyü azaltmak için morfolojik açma işlemi uygula
    cv.imshow("mask", mask) # Maskeyi göster
    contours, hierarchy = cv.findContours(mask,
                                          cv.RETR_EXTERNAL,
                                          cv.CHAIN_APPROX_SIMPLE) # Konturleri bul
    for c in range(len(contours)): # Tüm konturları sırayla işle
        area = cv.contourArea(contours[c]) # Konturun alanını hesapla
        if area < 150: # Eğer alan çok küçükse (önemsizse), işlemeye devam etme
            continue
        rect = cv.minAreaRect(contours[c]) # Konturu çevreleyen minimum dikdörtgeni bul
        cv.ellipse(image, rect, (0, 255, 0), 2, 8) # Dikdörtgenin çevresine elips çiz
        cv.circle(image, 
                  (np.int32(rect[0][0]), np.int32(rect[0][1])),
                   2, 
                   (255, 0 ,0), 
                   2, 
                   8, 
                   0) # Elipsin merkezine bir daire çiz
    return image # İşlenmiş görüntüyü döndür

In [23]:
while True: #işlem sürekli çalışsın diye
    ret, frame = cap.read() # Videodan bir kare oku, 
                            # 'ret' okuma başarılıysa True olur, 
                            # 'frame' ise alınan kare
    
    if not ret: # Eğer kare alınamadıysa döngüden çık
        print("Video sonuna ulaşıldı veya dosya okunamıyor.")
        break
    
    cv.imshow('input', frame)  # Orijinal video karesini göster
    
    result = process(frame) # İşlenmiş görüntüyü al
    
    cv.imshow("Result", result) # İşlenmiş görüntüyü göster

    k = cv.waitKey(10)&0xff  # 10 milisaniye bekle ve bir tuş girişini kontrol et
    if k ==27:               # Eğer 'Esc' tuşuna basılırsa (ASCII kodu 27), döngüden çık
        break

Video sonuna ulaşıldı veya dosya okunamıyor.


In [24]:
cv.destroyAllWindows() # Tüm pencereleri kapat

![](ss_of_videos_2.png)